In [2]:
import pandas as pd
from tqdm import tqdm


## Load Clusters and Alarms

In [46]:
real_time_clusters_df = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [47]:
real_time_clusters_df

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
39,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9
40,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13
41,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13


In [6]:
filtered_tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [5]:
filtered_tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [4]:
filtered_tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')


In [3]:
filtered_tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

In [2]:
filtered_mob_alarms_df = pd.read_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

## Extract Last Occurence field form Alarms

In [8]:
adsl_last_occurence = filtered_tx_alarms_adsl_df.loc[:, ['alarm_id', 'last_occurrence']]
pdh_last_occurence = filtered_tx_alarms_pdh_df.loc[:, ['alarm_id', 'last_occurrence']]
ptn_last_occurence = filtered_tx_alarms_ptn_df.loc[:, ['alarm_id', 'last_occurrence']]
sdh_last_occurence = filtered_tx_alarms_sdh_df.loc[:, ['alarm_id', 'last_occurrence']]
mob_last_occurence = filtered_mob_alarms_df.loc[:, ['alarm_id', 'last_occurrence']]


In [9]:
all_alarms = pd.concat([adsl_last_occurence, pdh_last_occurence, ptn_last_occurence, sdh_last_occurence, mob_last_occurence])

In [10]:
all_alarms.to_parquet('alarms datasets/alarms_last_occurence.parquet')

## Join Clusters with Last Occurence

In [48]:
all_alarms = pd.read_parquet('alarms datasets/alarms_last_occurence.parquet')

In [50]:
all_alarms

,alarm_id,last_occurrence
0,COS4P;86723842,2022-12-31 23:55:53
1,COS3P;66481069,2022-12-31 21:58:05
2,COS4P;86723883,2023-01-01 00:03:25
3,COS4P;86723880,2023-01-01 00:03:25
4,COS4P;86723881,2023-01-01 00:03:12
...,...,...
28176439,EFM;6591f1f639628d000137853b,2023-12-31 23:59:42
28176440,EFM;6591f1f739628d000137854e,2023-12-31 23:59:52
28176441,EFM;6591f1f739628d0001378554,2023-12-31 23:59:52
28176442,EFM;6591f1f939628d0001378585,2023-12-31 23:59:42


In [87]:
clusters_merge_df = pd.merge(real_time_clusters_df, all_alarms, on='alarm_id', how='left')

In [88]:
clusters_merge_df

,alarm_id,network,ne_type,first_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2,last_occurrence
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1,2023-01-02 13:52:56
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1,2023-01-02 13:52:56
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2,NaT
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3,2023-01-01 00:08:06
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4,2023-01-04 12:16:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123609,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13,2023-12-31 23:52:28
13123610,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9,2023-12-31 23:50:46
13123611,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13,2023-12-31 23:58:21
13123612,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13,2023-12-31 23:58:21


In [89]:
clusters_merge_df.insert(4, 'last_occurrence', clusters_merge_df.pop('last_occurrence'))
clusters_merge_df.to_parquet('real-time clusters/20230101-20240101_real_time_clusters_with_last_occurence.parquet')

In [90]:
clusters_merge_df

,alarm_id,network,ne_type,first_occurrence,last_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2023-01-02 13:52:56,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2023-01-02 13:52:56,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,NaT,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2023-01-01 00:08:06,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2023-01-04 12:16:06,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123609,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:52:28,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
13123610,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9
13123611,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:58:21,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13
13123612,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:58:21,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13


## Extract Clusters from Datasets

In [1]:
clusters_merge_df = pd.read_parquet('real-time clusters/20230101-20240101_real_time_clusters_with_last_occurence.parquet')

NameError: name 'pd' is not defined

In [68]:
clusters_merge_df['first_occurrence'] = pd.to_datetime(clusters_merge_df['first_occurrence'])
clusters_merge_df['last_occurrence'] = pd.to_datetime(clusters_merge_df['last_occurrence'])

In [69]:
clusters_merge_df

,alarm_id,network,ne_type,first_occurrence,last_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
0,EFM;63af9dfe2aaf2c000125807a,an,lnbts,2022-12-31 03:27:10,2023-01-02 13:52:56,2022-12-31 23:59:01,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
1,EFM;63af9dfec623b40001a5c9cf,an,lnbts,2022-12-31 03:27:10,2023-01-02 13:52:56,2022-12-31 23:58:55,2023-01-01 00:00:02,1btste08d,alad,2022-12-31 03:27:10,mob,None,202301010000_3,202301010000_1
2,COS1P;136902468,sdh,ad16,2022-12-31 22:19:34,NaT,2022-12-31 23:58:46,2023-01-01 00:00:02,alesitaj/ad16/73,ales,2022-12-31 22:19:34,tx,None,202301010000_20,202301010000_2
3,EFM;63b0beaf2aaf2c0001258736,an,lebts,2022-12-31 23:58:55,2023-01-01 00:08:06,2022-12-31 23:59:27,2023-01-01 00:00:02,5btsle119,alss,2022-12-31 23:58:55,mob,None,202301010000_9,202301010000_3
4,COS1P;136891403,sdh,oadm,2022-12-31 11:26:16,2023-01-04 12:16:06,2022-12-31 23:59:52,2023-01-01 00:00:02,ancoitab/oadm/1,anco,2022-12-31 11:26:16,tx,None,202301010000_21,202301010000_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123609,EFMTX;6591efb9d7d6bc00019444d8,adsl,adsl,2023-12-31 23:48:25,2023-12-31 23:52:28,2023-12-31 23:56:43,2023-12-31 23:58:02,taglitac/adsl/1,tagl,2023-12-31 23:48:24,tx,None,202312312350_16,202312312350_13
13123610,EFMTX;6591f046d8beb800016deaef,ptn,ptn3,2023-12-31 23:50:46,2023-12-31 23:50:46,2023-12-31 23:57:34,2023-12-31 23:58:02,thesitae/ptn3/1,thes,2023-12-31 23:44:44,tx,None,202312312348_17,202312312344_9
13123611,EFM;6591f199d4c961000177739d,an,lnbts,2023-12-31 23:56:25,2023-12-31 23:58:21,2023-12-31 23:56:37,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13
13123612,EFM;6591f19aba09b0000160c08d,an,lnbts,2023-12-31 23:56:26,2023-12-31 23:58:21,2023-12-31 23:57:59,2023-12-31 23:58:03,1btsts059,trie,2023-12-31 23:56:25,mob,None,202312312358_1,202312312358_13


In [70]:
columns = ['cluster_id', 'cluster_id2', 'alarm_id', 'first_occurrence', 'last_occurrence', 'network']

In [71]:
clusters_merge_df = clusters_merge_df[columns]

In [86]:
clusters_merge_df

,cluster_id,cluster_id2,alarm_id,first_occurrence,last_occurrence,network
0,202301010000_3,202301010000_1,EFM;63af9dfe2aaf2c000125807a,2022-12-31 03:27:10,2023-01-02 13:52:56,an
1,202301010000_3,202301010000_1,EFM;63af9dfec623b40001a5c9cf,2022-12-31 03:27:10,2023-01-02 13:52:56,an
3,202301010000_9,202301010000_3,EFM;63b0beaf2aaf2c0001258736,2022-12-31 23:58:55,2023-01-01 00:08:06,an
4,202301010000_21,202301010000_4,COS1P;136891403,2022-12-31 11:26:16,2023-01-04 12:16:06,sdh
5,202301010000_55,202301010000_5,EFM;63afc19f9603570001f34420,2022-12-31 05:59:11,2023-01-01 02:03:43,an
...,...,...,...,...,...,...
13123609,202312312350_16,202312312350_13,EFMTX;6591efb9d7d6bc00019444d8,2023-12-31 23:48:25,2023-12-31 23:52:28,adsl
13123610,202312312348_17,202312312344_9,EFMTX;6591f046d8beb800016deaef,2023-12-31 23:50:46,2023-12-31 23:50:46,ptn
13123611,202312312358_1,202312312358_13,EFM;6591f199d4c961000177739d,2023-12-31 23:56:25,2023-12-31 23:58:21,an
13123612,202312312358_1,202312312358_13,EFM;6591f19aba09b0000160c08d,2023-12-31 23:56:26,2023-12-31 23:58:21,an


In [83]:
cluster_nat = clusters_merge_df[clusters_merge_df["last_occurrence"].isnull()]

In [74]:
clusters_merge_df = clusters_merge_df.dropna(subset=["last_occurrence"])

In [81]:
clusters_merge_df

,cluster_id,cluster_id2,alarm_id,first_occurrence,last_occurrence,network
0,202301010000_3,202301010000_1,EFM;63af9dfe2aaf2c000125807a,2022-12-31 03:27:10,2023-01-02 13:52:56,an
1,202301010000_3,202301010000_1,EFM;63af9dfec623b40001a5c9cf,2022-12-31 03:27:10,2023-01-02 13:52:56,an
3,202301010000_9,202301010000_3,EFM;63b0beaf2aaf2c0001258736,2022-12-31 23:58:55,2023-01-01 00:08:06,an
4,202301010000_21,202301010000_4,COS1P;136891403,2022-12-31 11:26:16,2023-01-04 12:16:06,sdh
5,202301010000_55,202301010000_5,EFM;63afc19f9603570001f34420,2022-12-31 05:59:11,2023-01-01 02:03:43,an
...,...,...,...,...,...,...
13123609,202312312350_16,202312312350_13,EFMTX;6591efb9d7d6bc00019444d8,2023-12-31 23:48:25,2023-12-31 23:52:28,adsl
13123610,202312312348_17,202312312344_9,EFMTX;6591f046d8beb800016deaef,2023-12-31 23:50:46,2023-12-31 23:50:46,ptn
13123611,202312312358_1,202312312358_13,EFM;6591f199d4c961000177739d,2023-12-31 23:56:25,2023-12-31 23:58:21,an
13123612,202312312358_1,202312312358_13,EFM;6591f19aba09b0000160c08d,2023-12-31 23:56:26,2023-12-31 23:58:21,an


In [84]:
cluster_nat

,cluster_id,cluster_id2,alarm_id,first_occurrence,last_occurrence,network


In [14]:

clusters_grouped = clusters_merge_df.groupby(['cluster_id', 'cluster_id2'])


In [ ]:
clusters = []

# Itera sui gruppi e salva i cluster in una nuova struttura dati
for nome_gruppo, gruppo in gruppi:
    clusters.append(gruppo)

In [10]:
gruppi_count = gruppi.size().reset_index(name='size')

## Clusters Statistics

- Size: number of alarms within it
- Duration/lifespan: difference in time from the last last_occurrence to the first first_occurrence
- Delta first occurrence: distance between the last available first occurrence and the first ever first occurrence

In [11]:
gruppi_count.sort_values(by="size", ascending=False)

,cluster_id,cluster_id2,size
3313990,202311271242_16,202311271240_34,514
3209494,202311201646_7,202311201646_10,514
2994127,202311051440_21,202311051440_4,514
3643679,202312202238_5,202312202238_5,514
3593360,202312161922_20,202312161918_16,514
...,...,...,...
1561817,202307192116_6,202307192116_3,1
1561818,202307192116_7,202307192114_17,1
1561819,202307192116_8,202307192114_17,1
1561820,202307192116_9,202307192114_17,1


In [23]:
clusters[3593360]



,alarm_id,network,ne_type,first_occurrence,last_occurrence,last_update,data,ne_id,geo_code,min_cluster_first_occurrence,domain,link_cluster,cluster_id,cluster_id2
12710448,EFM;657dea652e6a8400010aca95,an,vnbts,2023-12-16 19:20:20,2023-12-16 19:21:36,2023-12-16 19:21:46,2023-12-16 19:22:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12710449,EFM;657dea652e6a8400010aca96,an,vnbts,2023-12-16 19:20:20,2023-12-16 19:21:36,2023-12-16 19:21:45,2023-12-16 19:22:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12710450,EFM;657dea652e6a8400010aca97,an,vnbts,2023-12-16 19:20:20,2023-12-16 19:21:36,2023-12-16 19:21:46,2023-12-16 19:22:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12710451,EFM;657dea652e6a8400010aca98,an,vnbts,2023-12-16 19:20:20,2023-12-16 19:21:36,2023-12-16 19:21:45,2023-12-16 19:22:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12710452,EFM;657dea652e6a8400010aca99,an,vnbts,2023-12-16 19:20:20,2023-12-16 19:21:36,2023-12-16 19:21:45,2023-12-16 19:22:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12711261,EFM;657dead62e6a8400010acc53,an,vnbts,2023-12-16 19:22:02,2023-12-16 19:24:56,2023-12-16 19:37:04,2023-12-16 19:38:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12711262,EFM;657dead72e6a8400010acc62,an,vnbts,2023-12-16 19:22:02,2023-12-16 19:24:46,2023-12-16 19:37:43,2023-12-16 19:38:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12711263,EFM;657dead92e6a8400010acc7f,an,vnbts,2023-12-16 19:22:02,2023-12-16 19:25:12,2023-12-16 19:37:58,2023-12-16 19:38:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16
12711264,EFM;657deada2e6a8400010acc80,an,vnbts,2023-12-16 19:22:02,2023-12-16 19:25:32,2023-12-16 19:37:59,2023-12-16 19:38:03,bbtsvi18d,soss,2023-12-16 19:20:20,mob,None,202312161922_20,202312161918_16


In [17]:
# Definire una funzione per calcolare la lifespan all'interno di ciascun gruppo
def calcola_lifespan(gruppo):
    first_occurrence_min = pd.to_datetime(gruppo['first_occurrence']).min()
    last_occurrence_max = pd.to_datetime(gruppo['last_occurrence']).max()
    return last_occurrence_max - first_occurrence_min

# Applica la funzione ai gruppi
lifespan_per_cluster = gruppi.apply(calcola_lifespan)

# Visualizza i risultati
print(lifespan_per_cluster)



C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_6292\538067073.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_occurrence_min = pd.to_datetime(gruppo['first_occurrence']).min()


C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_6292\538067073.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_occurrence_min = pd.to_datetime(gruppo['first_occurrence']).min()
C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_6292\538067073.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_occurrence_min = pd.to_datetime(gruppo['first_occurrence']).min()
C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_6292\538067073.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_occurrence_min = pd.to_datetime(gruppo['first_occurrence']).min(

KeyboardInterrupt: 